# TA-13 — Simulasi Predator–Prey (Lotka–Volterra)

Notebook ini menjalankan:
1. Load dataset `dataset.csv`
2. Normalisasi (penskalaan) data (dibagi 1000)
3. Visualisasi data asli
4. Simulasi model Lotka–Volterra dengan `odeint`
5. Overlay data asli vs hasil simulasi

> **Catatan penting:** Pastikan file `dataset.csv` berada **satu folder** dengan notebook ini.


## 0) Install library (sekali saja)

Jalankan cell ini **kalau** library belum terpasang.


In [ ]:
%pip install pandas numpy matplotlib scipy


## 1) Load dataset + normalisasi (penskalaan)

Sesuai tips dosen: kalau populasi besar, sebaiknya dibagi 1000 agar parameter model tidak terlalu kecil.


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("dataset.csv")

# rapikan nama kolom
df.columns = [c.strip().lower() for c in df.columns]

# cek kolom wajib
needed = {"year", "hare", "lynx"}
if not needed.issubset(set(df.columns)):
    raise ValueError(f"Kolom dataset harus punya {needed}, tapi yang ada: {set(df.columns)}")

# urutkan berdasarkan tahun
df = df.sort_values("year").reset_index(drop=True)

SCALE = 1000.0
df["hare_scaled"] = df["hare"] / SCALE
df["lynx_scaled"] = df["lynx"] / SCALE

df.head()


## 2) Visualisasi data asli (time series)

Garis biru = Hare (mangsa)  
Garis oranye = Lynx (predator)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(df["year"], df["hare_scaled"], label="Mangsa (Hare) / 1000")
plt.plot(df["year"], df["lynx_scaled"], label="Predator (Lynx) / 1000")
plt.xlabel("Tahun")
plt.ylabel("Populasi (skala ribuan)")
plt.title("Data Asli Predator–Prey (Hare vs Lynx) - Time Series")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


## 3) Definisi model Lotka–Volterra

\[
\frac{dx}{dt} = \alpha x - \beta xy
\]
\[
\frac{dy}{dt} = \delta xy - \gamma y
\]

x = mangsa (hare), y = predator (lynx)


In [ ]:
from scipy.integrate import odeint

def lotka_volterra(state, t, alpha, beta, gamma, delta):
    x, y = state
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y
    return [dxdt, dydt]


## 4) Simulasi model + overlay dengan data asli

- Data asli: garis putus-putus  
- Model: garis solid  

Kamu bisa ubah parameter di cell ini untuk mencoba tuning.


In [ ]:
# data (sudah diskalakan)
hare_data = df["hare_scaled"].values
lynx_data = df["lynx_scaled"].values

# waktu simulasi: 1 langkah = 1 tahun, sepanjang dataset
t = np.arange(len(df))

# parameter awal (boleh kamu ubah untuk tuning)
alpha = 0.6
beta  = 0.02
gamma = 0.5
delta = 0.01

# kondisi awal dari tahun pertama
initial_state = [hare_data[0], lynx_data[0]]

# simulasi
solution = odeint(
    lotka_volterra,
    initial_state,
    t,
    args=(alpha, beta, gamma, delta)
)

hare_sim = solution[:, 0]
lynx_sim = solution[:, 1]

# plot overlay
plt.figure(figsize=(11, 6))

plt.plot(t, hare_data, "b--", label="Hare (Data Asli)")
plt.plot(t, lynx_data, "r--", label="Lynx (Data Asli)")

plt.plot(t, hare_sim, "b", label="Hare (Model)")
plt.plot(t, lynx_sim, "r", label="Lynx (Model)")

plt.xlabel("Waktu (tahun)")
plt.ylabel("Populasi (skala ribuan)")
plt.title("Model Lotka–Volterra vs Data Asli")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## 5) Catatan untuk deployment Streamlit (opsional)

Notebook ini fokus untuk laporan/eksperimen.  
Untuk aplikasi interaktif (slider), kamu sudah punya `app.py` Streamlit yang dideploy ke Railway.
